### OGR Basics

OGR offers the following seven classes:

1. Geometry
1. Spatial Reference
1. Feature
1. Feature Class Definition
1. Layer
1. Dataset
1. Drivers

In [2]:
from osgeo import ogr, osr
import os

List the attributes of a shapefile

In [3]:
shapefile = 'data/census/tl_2019_06_tract.shp'
source = ogr.Open(shapefile)
layer = source.GetLayer()
layer_def = layer.GetLayerDefn()
schema = []
for n in range(layer_def.GetFieldCount()):
    field_def = layer_def.GetFieldDefn(n)
    schema.append(field_def.name)
print(schema)

['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON']


Get the number of features in a shapefile

In [4]:
driver = ogr.GetDriverByName('ESRI Shapefile')
data_source = driver.Open(shapefile, 0)
lyr = data_source.GetLayer()
feature_count = lyr.GetFeatureCount()
print(f'Number of features in {os.path.basename(shapefile)}: {feature_count}')

Number of features in tl_2019_06_tract.shp: 8057


Get the Spatial Reference from a layer or geometry

In [14]:
spatial_ref = layer.GetSpatialRef()
print(spatial_ref)

GEOGCS["NAD83",
    DATUM["North_American_Datum_1983",
        SPHEROID["GRS 1980",6378137,298.257222101,
            AUTHORITY["EPSG","7019"]],
        AUTHORITY["EPSG","6269"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4269"]]


In [16]:
feature = layer.GetNextFeature()
geom = feature.GetGeometryRef()
geom_spatial_ref = geom.GetSpatialReference()
print(geom_spatial_ref)

GEOGCS["NAD83",
    DATUM["North_American_Datum_1983",
        SPHEROID["GRS 1980",6378137,298.257222101,
            AUTHORITY["EPSG","7019"]],
        AUTHORITY["EPSG","6269"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4269"]]


Get the centroid of a geometry

In [ ]:
for feature in layer:
    geom = feature.GetGeometryRef()
    print(geom.Centroid().ExportToWkt())

##### Creating a geometry

Adding the coordinates directly

In [20]:
geom = ogr.Geometry(ogr.wkbLinearRing)
geom.AddPoint(1, 1)
geom.AddPoint(5, 1)
geom.AddPoint(5, 5)
geom.AddPoint(1, 5)
geom.AddPoint(1, 1)
polygon = ogr.Geometry(ogr.wkbPolygon)
polygon.AddGeometry(geom)
print(polygon.ExportToWkt())

POLYGON ((1 1 0,5 1 0,5 5 0,1 5 0,1 1 0))


from GeoJSON

In [8]:
geojson = """{ "type": "Polygon", "coordinates": [[[1, 1], [5, 1], [5, 5], [1, 5], [1, 1]]]}"""
polygon = ogr.CreateGeometryFromJson(geojson)
print(polygon)

POLYGON ((1 1,5 1,5 5,1 5,1 1))


#### Basic Operations

Area of a polygon

In [23]:
# Area of a polygon
polygon.Area()

16.0

In [29]:
# Centroid
print(polygon.Centroid().ExportToWkt())

POINT (3 3)


In [30]:
# Boundary
print(polygon.GetBoundary().ExportToWkt())

LINESTRING (1 1,5 1,5 5,1 5,1 1)


In [31]:
# Convex hull
print(polygon.ConvexHull().ExportToWkt())

POLYGON ((1 1,1 5,5 5,5 1,1 1))


In [35]:
# Buffer
buffer = polygon.Buffer(distance=0)
print(buffer.ExportToWkt())

POLYGON ((1 1,1 5,5 5,5 1,1 1))


In [36]:
# Contains
point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(10, 10)
print(polygon.Contains(point))

False


#### Writing data to a new shapefile

In [5]:
# Defining the spatial reference for the new data
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
spatial_ref

<osgeo.osr.SpatialReference; proxy of <Swig Object of type 'OSRSpatialReferenceShadow *' at 0x7f8834aa8330> >

In [6]:
# Creating the layer
driver = ogr.GetDriverByName('ESRI Shapefile')
output_shp = driver.CreateDataSource('output/output_polygon_test.shp')
layer = output_shp.CreateLayer('Output', spatial_ref, ogr.wkbPolygon)
layer_def = layer.GetLayerDefn()
print(layer_def)

<osgeo.ogr.FeatureDefn; proxy of <Swig Object of type 'OGRFeatureDefnShadow *' at 0x7f8834aa83f0> >


In [9]:
# Creating a feature
feature_index = 0
feature = ogr.Feature(layer_def)
feature.SetGeometry(polygon)
feature.SetFID(feature_index)
print(feature)

<osgeo.ogr.Feature; proxy of <Swig Object of type 'OGRFeatureShadow *' at 0x7f88604e8c60> >


In [43]:
layer.CreateFeature(feature)

0

#### Using a spatial filter to select features

In [10]:
extent = lyr.GetExtent()
print(extent)
# minx, miny, maxx, maxy
lyr.SetSpatialFilterRect(extent[0] + 1, extent[1] + 1, extent[2] - 10, extent[3] - 9.41)
print(f'Total features: {lyr.GetFeatureCount()}')

layer_def = lyr.GetLayerDefn()
for index in range(layer_def.GetFieldCount()):
    field_def = layer_def.GetFieldDefn(index)
    print(field_def.name)
field_def = layer_def.GetFieldDefn(0)
# print(field_def.name)


for feature in lyr:
    if feature.GetField('TRACTCE').startswith('01'):
        continue
    else:
        geom = feature.GetGeometryRef()
        print(f'Centroid: {geom.Centroid().ExportToWkt()}')
        print(f'Field value: {feature.GetField("TRACTCE")}')

(-124.48200299999999, -114.131211, 32.528832, 42.009502999999995)
Total features: 38
STATEFP
COUNTYFP
TRACTCE
GEOID
NAME
NAMELSAD
MTFCC
FUNCSTAT
ALAND
AWATER
INTPTLAT
INTPTLON
Geometry: POINT (-117.330314237958 32.9304065070288)
Field value: 990100
Geometry: POINT (-116.779177059395 32.6667234678629)
Field value: 021302
Geometry: POINT (-116.438560708022 32.7043953032112)
Field value: 021100
